# **Construindo um Chatbot com a API Gemini, Fundamentado em Documentos de texto**

**Além do Conhecimento Geral para uma IA Fundamentada**
A tarefa de criar um chatbot que responda a perguntas de usuários é uma aplicação comum de Grandes Modelos de Linguagem (LLMs). No entanto, um desafio significativo com LLMs de propósito geral, como o Gemini, é sua tendência a "alucinar".

Para aplicações empresariais, acadêmicas ou especializadas, onde a precisão e a rastreabilidade são primordiais, as respostas do chatbot devem ser estritamente limitadas a um conjunto de documentos.

A solução para este desafio é uma arquitetura conhecida como **Geração Aumentada por Recuperação (RAG)**. Em vez de simplesmente fazer uma pergunta ao LLM e esperar que ele "saiba" a resposta com base em seu vasto treinamento prévio, o RAG introduz um passo intermediário crucial.

![](https://static.wixstatic.com/media/cfe500_0546c1c5b8b3430f90c039aaa4ab71e2~mv2.jpg/v1/fill/w_740,h_438,al_c,q_80,usm_0.66_1.00_0.01,enc_avif,quality_auto/cfe500_0546c1c5b8b3430f90c039aaa4ab71e2~mv2.jpg)

1. **Prompt do Usuário**: Primeiro, o usuário insere um prompt com a pergunta (query) no sistema.

2. **Busca por Informação**: A query é usada para consultar fontes de conhecimento (como documentos PDF, TXT, etc.) previamente indexadas.

3. **Recuperação**: O sistema retorna ***trechos relevantes dos documentos*** que servirão como ***contexto aumentado***.

4. **Preparação do Prompt**: O contexto recuperado é combinado com a query original, formando um novo prompt fundamentado.

5. **Geração da Resposta**: O novo prompt é enviado ao LLM (como o Gemini), que gera uma resposta com base exclusivamente nas informações recuperadas.

A arquitetura RAG pode ser definida como um pipeline de três estágios principais: **Indexação, Recuperação e Geração**.


## 1. **Indexação**

A Indexação na pratica é um processo em 3 etapas:

1.1 **Ingestão**: Carregar e analisar os arquivos brutos PDF e TXT do seu corpus de conhecimento.


Instalação das dependencias:
* google-generativeai: O SDK oficial do Google para interagir com a API Gemini.

Chave da API do Gemini

In [ ]:
GOOGLE_API_KEY = ""

 Importando texto de PDF

 Para este exemplo usaremos a [Organização didática do IFPI](https://www.ifpi.edu.br/acesso-a-informacao/institucional/consuprn1432022organizacaodidatica.pdf).

 Baixe o arquivo do link.

Instalação das dependencias:
* pypdf: Biblioteca para extrair texto de arquivos PDF.

In [ ]:
!pip install -U -q pypdf

1.2 **Divisão (Chunking)**: Dividir estrategicamente o texto dos documentos em pedaços menores e gerenciáveis, conhecidos como "chunks".

Esta etapa é fundamental por duas razões principais:

 - **Limites de Contexto do Modelo**: Os modelos de embedding e os LLMs têm um limite máximo de tokens que podem processar de uma só vez. Enviar um documento inteiro (100 páginas, por exemplo) excederia esse limite.

- **Precisão da Recuperação**: Se você incorporar um documento inteiro em um único vetor, o vetor representará o significado médio de todo o documento. Quando um usuário faz uma pergunta específica, é improvável que a média de todo o documento seja a correspondência mais próxima. Chunks menores e mais focados permitem uma recuperação muito mais precisa e relevante.

A escolha da estratégia de "chunking" é uma decisão de design importante. As estratégias comuns incluem:

- **Tamanho Fixo (Fixed-Size)**: A abordagem mais simples, dividindo o texto em chunks de N caracteres ou tokens. Sua principal desvantagem é que pode cortar frases ou parágrafos no meio, quebrando o contexto semântico.
- **Semântica (Semantic)**: Tenta dividir o texto em limites lógicos, como frases ou parágrafos.
- **Recursiva (Recursive)**: Uma abordagem mais sofisticada que tenta dividir o texto usando uma hierarquia de separadores. Por exemplo, primeiro tenta dividir por parágrafos (\n\n). Se os chunks resultantes ainda forem muito grandes, ele os divide por frases, e assim por diante.

Para a maioria dos documentos baseados em texto, a **Divisão Recursiva de Caracteres (Recursive Character Text Splitting)** oferece o melhor equilíbrio entre simplicidade e preservação do contexto semântico. Ela respeita a estrutura natural do documento, tentando manter os parágrafos e as frases intactos sempre que possível.

Dois parâmetros chave nesta estratégia são `chunk_size` e `chunk_overlap`. `chunk_size` define o tamanho máximo de cada chunk. `chunk_overlap` especifica quantos caracteres do final de um chunk devem ser repetidos no início do próximo.

![chunk overlap](https://cdn.analyticsvidhya.com/wp-content/uploads/2025/02/unnamed-1-67a0e0c9ca199-1.webp)

Instalação das dependencias:
* langchain-text-splitters: Uma ferramenta útil para implementar estratégias de divisão de texto.

In [ ]:
!pip install -U -q langchain-text-splitters

1.3 **Embedding e Indexação**: Converter cada "chunk" de texto em uma representação numérica (um vetor de embedding) e armazenar esses vetores para busca rápida.

 - *Embedding:*

- *Indexação:*

Os embeddings precisam ser armazenados em um local que permita uma busca por similaridade extremamente rápida. Bancos de dados relacionais tradicionais não são projetados para este tipo de operação. Em vez disso, usamos **armazenamentos de vetores (vector stores)**. Seguem duas opções:
1. **[ChromaDB](https://www.trychroma.com/)** é um banco de dados vetorial de código aberto, construído especificamente para aplicações de IA. Ele abstrai grande parte da complexidade. Com uma API simples, ele gerencia o armazenamento de embeddings, documentos e metadados em um único local.  

2. **[FAISS (Facebook AI Similarity Search)](https://ai.meta.com/tools/faiss/)** não é um banco de dados, mas sim uma biblioteca de código aberto altamente otimizada para busca de similaridade em conjuntos densos de vetores.

Para os propósitos deste guia, que visa construir um protótipo funcional, **ChromaDB** é a escolha recomendada devido à sua simplicidade e ciclo de desenvolvimento rápido.

Instalação das dependencias:
* ChromaDB: Um banco de dados vetorial de código aberto projetado para IA.

In [ ]:
!pip install -U -q chromadb

Adiciona os dados à coleção (como um insert em uma tabela)

## 2. **Recuperação**

Quando um usuário envia uma consulta, o primeiro passo é encontrar os "chunks" de texto mais relevantes em nossa base de conhecimento. Este processo de busca semântica tem duas etapas:

1. **Incorporar a Consulta do Usuário**: A consulta do usuário (uma string de texto) deve ser convertida em um vetor de embedding usando o mesmo modelo que usamos para os documentos (text-embedding-004). Crucialmente, aqui usamos `task_type="RETRIEVAL_QUERY"` para otimizar o vetor para a tarefa de busca.

2. **Buscar por Similaridade**: O vetor da consulta é então usado para pesquisar no nosso armazenamento de vetores. A "similaridade" é tipicamente medida usando a **Similaridade de Cosseno (Cosine Similarity)**. Esta métrica calcula o cosseno do ângulo entre dois vetores. Um valor de `1` significa que os vetores apontam na mesma direção (semanticamente idênticos), `0` significa que são ortogonais (não relacionados), e `-1` significa que são opostos. Matematicamente, é calculado como o produto escalar dos vetores dividido pelo produto de suas magnitudes:
$$
S_C(A, B) = \frac{A \cdot B}{\|A\| \|B\|}
$$

## 3. **Geração**

Nesta etapa vamos usar os "chunks" recuperados como contexto para um modelo generativo.

Esta é a etapa mais crítica para cumprir a restrição principal do usuário: garantir que o chatbot responda apenas com base nas informações fornecidas. Devemos construir um prompt que instrua o modelo Gemini a abandonar seu conhecimento geral e a se ater estritamente ao contexto que fornecemos. Isso é alcançado através de uma **engenharia de prompt**.

**Resposta Final**

**Chat continuo**